In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_EPIR_Pair1", "ET2p01_BAR_4", "ET2p01_BAR_6", "ET2_EPIR_Pair4"]
chip_fignames = ["ET2 EPIR Pair 1", "ET2.01 Barcelona 4", "ET2.01 Barcelona 6", "ET2 EPIR Pair4"]
# chip_names = ["ET2_EPIR_Pair4"]
# chip_fignames = ["EPIR Pair 4"]
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM1"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60, 0x61, 0x62, 0x64]
# chip_addresses = [0x63]
ws_addresses = [None, None, None, None]

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

col_list, row_list = np.meshgrid(np.arange(16),np.arange(16))
scan_list = list(zip(row_list.flatten(),col_list.flatten()))

# Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,chip_fignames,[("1","1"),("1","1"),("1","1"), ("1","1")])

# Config chips

### Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [ ]:
# (WS Prep Pixel and Peri) - 0 - (disable & auto_cal all pixels) - (auto_TH_CAL) - (disable default all pixels) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
i2c_conn.config_chips('00101101')
# i2c_conn.config_chips('00001111') # disable and more fun things
# i2c_conn.config_chips('00000100') # set basic peripherals
# i2c_conn.config_chips('00010000') # calibrate only
# i2c_conn.config_chips('00000001') # pixel ID check (I2C check)
# i2c_conn.config_chips('10000000') # ws init
# i2c_conn.disable_all_pixels(chip_address=chip_addresses[1])

## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
# def make_baselines(chip_addresses, chip_names):
#     for chip_address, chip_name in zip(chip_addresses, chip_names):
#         i2c_conn.auto_calibration(chip_address=chip_address,chip_name=chip_name)

#     histdir = Path('/media/daq/X9/ETROC-History-DESY-Feb-2024/')
#     histdir.mkdir(exist_ok=True)
#     histfile = histdir / 'BaselineHistory.sqlite'
#     i2c_conn.save_baselines(chip_fignames,fig_path,histdir,histfile)

#     time.sleep(5)

def make_baselines(do_config=True):
    if(do_config):
        i2c_conn.config_chips('00100101')

    histdir = Path('/media/daq/X9/ETROC-History-DESY-Feb-2024/')
    histdir.mkdir(exist_ok=True)
    histfile = histdir / 'BaselineHistory.sqlite'
    i2c_conn.save_baselines(chip_fignames,fig_path,histdir,histfile,show_BLs=False)

    time.sleep(5)

In [ ]:
def do_pixel_operations(board_offsets=None, noisy_pixels=None, power_mode='high'):
    i2c_conn.enable_select_pixels_in_chips(scan_list,QInjEn=False,Bypass_THCal=True,specified_addresses=chip_addresses[:], power_mode=power_mode)

    # offset = 0x0f
    # for chip_address in chip_addresses[:]:
    #     chip = i2c_conn.get_chip_i2c_connection(chip_address)
    #     row_indexer_handle,_,_ = chip.get_indexer("row")
    #     column_indexer_handle,_,_ = chip.get_indexer("column")
    #     for row, col in scan_list:
    #         print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
    #         column_indexer_handle.set(col)
    #         row_indexer_handle.set(row)
    #         i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)
    #     del chip, row_indexer_handle, column_indexer_handle

    if board_offsets is not None:
        time.sleep(5)

        for chip_address,chip_name in zip(chip_addresses,chip_names):
            if chip_address not in board_offsets:
                continue
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            i2c_conn.set_chip_offsets(chip_address, chip_name, offset=board_offsets[chip_address], chip=chip, pixel_list=scan_list, verbose=False)
            # row_indexer_handle,_,_ = chip.get_indexer("row")
            # column_indexer_handle,_,_ = chip.get_indexer("column")
            # for row, col in scan_list:
            #     print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col}) Setting Offset to {board_offsets[chip_address]}")
            #     column_indexer_handle.set(col)
            #     row_indexer_handle.set(row)
            #     # i2c_conn.pixel_decoded_register_write("QSel", format(0x14, '05b'), chip)
            #     i2c_conn.pixel_decoded_register_write("TH_offset", format(board_offsets[chip_address], '06b'), chip)
            # del chip, row_indexer_handle, column_indexer_handle
            del chip

    if noisy_pixels is not None:
        time.sleep(5)

        for chip_address in chip_addresses:
            if chip_address not in noisy_pixels:
                continue
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            row_indexer_handle,_,_ = chip.get_indexer("row")
            column_indexer_handle,_,_ = chip.get_indexer("column")
            for row, col in noisy_pixels[chip_address]:
                print(f"Masking from trigger of chip {hex(chip_address)} Pixel ({row},{col})")
                column_indexer_handle.set(col)
                row_indexer_handle.set(row)
                i2c_conn.pixel_decoded_register_write("disTrigPath", "1", chip)
                #i2c_conn.pixel_decoded_register_write("DAC", format(1023, '010b'), chip)
                #i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", chip)

    time.sleep(5)

In [ ]:
readout_board_names = "b0_EPIR_Pair1_b1_BAR_4_b2_BAR_6_b3_EPIR_Pair4"
self_trigger_delay = 484
beam_downtime = False # default is False because we expect (hope?) for 100% uptime from accelerator
beam_off_time = "10-00" # time that beam is expected to turn off
beam_on_time = "11-30" # time that beam is expected to turn on
info = "offset_15" # additional info for the output name
LED_pages = ["0x0000","0x0004","0x0008","0x000C"] # LED pages to cycle through
active_channels = "0x000f"
polarity = "0x0123" # "0x0023" or "0x0027"
hostname = "192.168.2.3"
ssd_path = "/media/daq/X9/"

In [ ]:
def run_daq(trigger_board, trigger_board_name, run_number, total_time, daq_time, run_options="--compressed_binary --skip_translation"):
    trigger_bit_delay = int(trigger_board+'11'+format(self_trigger_delay, '010b'), base=2)
    process_time = daq_time + 5
    iterations = ceil(total_time/process_time)
    LED_id = 0

    for it in range(iterations):
        if beam_downtime:
            current_time = datetime.datetime.now().strftime("%H-%M")
            while ((current_time > beam_off_time) and (beam_on_time >= current_time)):
                print("Waiting for beam...")
                time.sleep(300)
                current_time = datetime.datetime.now().strftime("%H-%M")
        LED_page = LED_pages[LED_id]
        LED_id += 1
        if LED_id >= len(LED_pages): LED_id = 0
        reset_string = ""
        if(it==0): reset_string = "--check_valid_data_start"
        parser = parser_arguments.create_parser()
        (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t {process_time} --run_name Run_{run_number} -o loop_{it} -v -w -s {LED_page} -p {polarity} -d {trigger_bit_delay} -a {active_channels} {run_options} --start_DAQ_pulse --stop_DAQ_pulse --ssd_path {ssd_path} {reset_string}".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Run_{run_number}_loop_{it}'))
        process.start()

        IPC_queue.put('memoFC Start Triggerbit BCR')
        while not IPC_queue.empty():
            pass
        time.sleep(daq_time)
        IPC_queue.put('stop DAQ')
        IPC_queue.put('memoFC Triggerbit')
        while not IPC_queue.empty():
            pass
        IPC_queue.put('allow threads to exit')
        process.join()

        del IPC_queue, process, parser

In [ ]:
run_numbers = [0]
board_offsets = {
    0x60: 0x0f,
    0x61: 0x0f,
    0x62: 0x0f,
    0x63: 0x0f,
}
noisy_pixels = {
    0x60: [ # (row, col)
        (10, 8),
        (13, 4),
    ],
}
trigger_board = '0001'
trigger_board_name = "EPIR_Pair1"
power_mode = 'high'
total_time = 1 * 60 * 60
daq_time = 300
BL_chip_addresses = chip_addresses
BL_chip_names = chip_names


for run_number in run_numbers:
    make_baselines(BL_chip_addresses,BL_chip_names)
    do_pixel_operations(board_offsets=board_offsets, noisy_pixels=noisy_pixels, power_mode=power_mode)
    print(f"Run {run_number} started at:", datetime.datetime.now().isoformat(sep=" "))
    run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number=run_number, total_time=total_time, daq_time=daq_time)
    print(f"Run {run_number} ended at:", datetime.datetime.now().isoformat(sep=" "))